# Deep Belief Network
By Hunter McGee

*Using yfinance from Cruz Jean for loading data and preprocessing.*

*DBN model and inspiration from: https://github.com/albertbup/deep-belief-network*


In [98]:
import keras
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

In [99]:
def getstock(name):
    t = yf.Ticker(name)
    dat = t.history(period='max')
    return np.array(dat['Open'][-3800:])

In [100]:
stocks = ['PFE','DIS','VZ','PG','ADBE','INTC','AAPL','MSFT','GOOG']
choice = 4 #index of stock to use

In [101]:
yf.Ticker(stocks[choice]).history(period='max')

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
1986-08-13,0.00,0.16,0.15,0.15,18899200,0.0,0.0
1986-08-14,0.00,0.17,0.16,0.16,4160000,0.0,0.0
1986-08-15,0.00,0.16,0.16,0.16,4332800,0.0,0.0
1986-08-18,0.00,0.16,0.15,0.15,2828800,0.0,0.0
1986-08-19,0.00,0.16,0.16,0.16,2060800,0.0,0.0
...,...,...,...,...,...,...,...
2020-03-25,308.00,317.65,298.13,305.91,4830400,0.0,0.0
2020-03-26,305.99,324.43,305.69,322.67,4531000,0.0,0.0
2020-03-27,311.70,314.29,300.63,305.83,4862100,0.0,0.0


In [102]:
# Grab full data set for stock
data = np.array([getstock(stocks[choice])])


In [103]:
window_width = 128

In [104]:
norm_factor_prices = np.array([data[:,i] for i in range(data.shape[1] - window_width-1)])
norm_factor_prices.shape


(3671, 1)

In [105]:
x = np.array([data[:,i:i+window_width] / data[:,i,None] for i in range(data.shape[1] - window_width - 1)])
x.shape

(3671, 1, 128)

In [106]:
# drop middle dimension
x_new= x[:,0,:]
x_new.shape

(3671, 128)

In [107]:
# Notice that the windows are normalized according to the first day of the window.
# That means that each data point shows the change relative to that first day.
x

array([[[1.        , 1.00097624, 1.00943703, ..., 0.87796941,
         0.90855841, 0.89424016]],

       [[1.        , 1.00845254, 1.00682705, ..., 0.9076723 ,
         0.89336801, 0.88654096]],

       [[1.        , 0.99838814, 1.00676983, ..., 0.88588008,
         0.87911025, 0.87814313]],

       ...,

       [[1.        , 1.01322799, 0.98539485, ..., 1.07516399,
         1.17044178, 1.11622513]],

       [[1.        , 0.97253022, 0.99599399, ..., 1.15516131,
         1.10165248, 1.09446312]],

       [[1.        , 1.02412652, 1.04681868, ..., 1.1327694 ,
         1.12537698, 1.14637734]]])

In [108]:
y = np.array([data[:,i+window_width] / data[:,i] for i in range(data.shape[1] - window_width-1)])
y = y[:,:]
y.shape


(3671, 1)

### Deep Belief Network

In [109]:
#from sklearn.neural_network import BernoulliRBM # for Restricted Boltzmann Machine units


from dbn import SupervisedDBNRegression

In [110]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_new, y, test_size=0.10)

In [111]:
x_test.shape

(368, 128)

### Train Model
If you notice RBM error or ANN loss not updating much between epochs, then retrain. (seems to be a gradient issue?)

In [112]:
# Train Model
model = SupervisedDBNRegression(hidden_layers_structure=[128],
                               learning_rate_rbm=0.015,
                               learning_rate=0.010,
                               n_epochs_rbm=12,
                               n_iter_backprop=10,
                               batch_size=8,
                               activation_function='relu')

model.fit(x_train, y_train)

# If you notice RBM error or ANN loss not updating much between epochs,
# then retrain. (seems to be a gradient issue?)


[START] Pre-training step:
>> Epoch 1 finished 	RBM Reconstruction error 5651.790236
>> Epoch 2 finished 	RBM Reconstruction error 1621.566099
>> Epoch 3 finished 	RBM Reconstruction error 399.435829
>> Epoch 4 finished 	RBM Reconstruction error 2109.874647
>> Epoch 5 finished 	RBM Reconstruction error 5681.546445
>> Epoch 6 finished 	RBM Reconstruction error 90.572058
>> Epoch 7 finished 	RBM Reconstruction error 2317.340481
>> Epoch 8 finished 	RBM Reconstruction error 121.291219
>> Epoch 9 finished 	RBM Reconstruction error 2220.626763
>> Epoch 10 finished 	RBM Reconstruction error 524.676712
>> Epoch 11 finished 	RBM Reconstruction error 684.411520
>> Epoch 12 finished 	RBM Reconstruction error 189.307384
[END] Pre-training step
[START] Fine tuning step:
>> Epoch 1 finished 	ANN training loss 0.028398
>> Epoch 2 finished 	ANN training loss 0.015501
>> Epoch 3 finished 	ANN training loss 0.013691
>> Epoch 4 finished 	ANN training loss 0.012142
>> Epoch 5 finished 	ANN training loss 

SupervisedDBNRegression()

### Test and RMSE
Uncomment print line in for loop to see predictions at each time step

In [114]:
# TEST SET ERROR
# Uncomment print line in for loop to see predictions
# at each time step
sum_squares = 0
for i in range(0,y_test.shape[0]):
    y_pred = model.predict(x_test[i,:])
    sum_squares += np.power(np.abs(y_pred - y_test[i]),2)
    #print('Predicted -> \t', y_pred, y_test[i],'\t<- Expected')
    #fe = (y_pred*norm_factor_prices[i]) - (y_test[i]*norm_factor_prices[i])
    #sum_squares += np.power(np.abs(fe),2)
    #print('Predicted -> \t', y_pred*norm_factor_prices[i], y_test[i]*norm_factor_prices[i],'\t<- Expected || FE: ',fe)


rmse = np.sqrt(sum_squares / y_test.shape[0])
print("RMSE: ", rmse)

RMSE:  [[0.08178752]]
